In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [4]:
import pandas as pd
from datetime import datetime


pos = 2
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos
2,amanda,05067676477,sobreira


In [5]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [6]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

    children = nav.find_element(By.ID, 'maxAge')
    nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
    time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [7]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                traceback.print_exc()
                
            if NoSuchElementException: 
                traceback.print_exc() 
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [8]:
def get_individual_info():
    
    individual = get_table()
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], []
    
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        
        if i_cns[0].upper() != 'CNS':
            cns.append('--')
        else:
            cns.append(i_cns[1])
            
            
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')
    
    nav.quit()
    
    return df_individual

# indi = get_indivivual_info()

In [9]:
children = get_individual_info()


Extração iniciada as: 2024-02-06 15:14:15.651267
Iniciando lista de links: 2024-02-06 15:14:32.536999


Traceback (most recent call last):
  File "/tmp/ipykernel_51576/2182560134.py", line 23, in get_table
    page.click()
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 80, in click
    self._execute(Command.CLICK_ELEMENT)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 633, in _execute
    return self._parent.execute(command, params)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.ElementClickInterceptedException: Message: Element <a class="page-link" href="#"> is not clickable at point (1055,446) because another element <li id="individuo-list-d

acabaram as paginas
lista de links finalizada as: 2024-02-06 15:15:26.700012
Iniciando individuos : 2024-02-06 15:15:26.700073
Finalizando individuos : 2024-02-06 15:35:33.786150
Extração finalizada as: 2024-02-06 15:35:33.787229


In [10]:
# display(children)
# children.to_excel('teste.xlsx', index=False)
children = children[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]

In [11]:
display(children)
children.to_excel(f'{data_e_hora}_{name}_children.xlsx', index=False)

,cpf,name,b_date,sex,race,m_name,link
0,17447854492,ADAM KHALIL SANTOS DA SILVA,06/12/2020,MASCULINO,BRANCA,LUCIENE GOMES DOS SANTOS,https://afogadosdaingazeira.ephealth.com.br/po...
1,18553130495,ADAM MIGUEL TEODORO DA SILVA,28/05/2023,FEMININO,PARDA,DAIRLA MARIA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
2,00093170408,AGATHA GABRIELLE HONORATO OLIVEIRA,05/11/2023,FEMININO,BRANCA,VALDILECIA HONORATO ALVES OLIVEIRA,https://afogadosdaingazeira.ephealth.com.br/po...
3,16565478481,AGHATA LORENA DE LIMA CAMPOS,28/06/2019,FEMININO,PARDA,ALINE DA SILVA CAMPOS,https://afogadosdaingazeira.ephealth.com.br/po...
4,18202556465,AGUIDA MAÍSA SILVA OLIVEIRA,23/06/2022,FEMININO,BRANCA,AGUIDA PALOMA DE SOUZA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...
273,18562672459,YARA SOFIA SOUZA DE MEDEIROS,25/05/2023,FEMININO,PARDA,MARIA AMANDA SOUZA VITORINO,https://afogadosdaingazeira.ephealth.com.br/po...
274,12386399508,YKARO EMMANUEL SANTOS OLIVEIRA,26/03/2021,MASCULINO,BRANCA,BRUNA SANTOS DE JESUS,https://afogadosdaingazeira.ephealth.com.br/po...
275,17239721426,YSIS BEATRIZ RIBEIRO ALVES,22/06/2020,FEMININO,PARDA,ALINE RIBEIRO MARTINS,https://afogadosdaingazeira.ephealth.com.br/po...
276,17265887425,YSIS EDUARDA MARQUES LOPES,15/07/2020,FEMININO,PARDA,ILDA VANESSA MARQUES,https://afogadosdaingazeira.ephealth.com.br/po...
